In [24]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
from joblib import Parallel, delayed
from rake_nltk import Rake

In [25]:
base_url='https://www.goodreads.com'

In [26]:
r = requests.get('https://www.goodreads.com/list/show/3810.Best_Cozy_Mystery_Series?page=1')
soup = BeautifulSoup(r.text, 'html.parser')

In [27]:
link = soup.find('a', class_='bookTitle')
href = link['href']
print(href)

/book/show/16331.Murder_at_the_Vicarage


In [28]:

product_links = set()

for x in range(0,2):  
    r = requests.get(f'https://www.goodreads.com/list/show/3810.Best_Cozy_Mystery_Series?page={x}')
    soup = BeautifulSoup(r.text, 'html.parser')
    
    product_list = soup.find_all('a', class_='bookTitle', href=True)
    
    for link in product_list:
        product_links.add(base_url + link['href'])

product_links = list(product_links)


In [29]:
product_links

['https://www.goodreads.com/book/show/16361.The_Secret_of_Chimneys',
 'https://www.goodreads.com/book/show/352427.Mum_s_the_Word',
 'https://www.goodreads.com/book/show/1176884.Body_Movers',
 'https://www.goodreads.com/book/show/140096.The_Julius_House',
 'https://www.goodreads.com/book/show/7039.Tears_of_the_Giraffe',
 'https://www.goodreads.com/book/show/338691.Still_Life',
 'https://www.goodreads.com/book/show/363613.Pick_Your_Poison',
 'https://www.goodreads.com/book/show/140090.Three_Bedrooms_One_Corpse',
 'https://www.goodreads.com/book/show/586437.The_Tale_of_Hill_Top_Farm',
 'https://www.goodreads.com/book/show/45020.Shakespeare_s_Landlord',
 'https://www.goodreads.com/book/show/33442.Sugar_Cookie_Murder',
 'https://www.goodreads.com/book/show/638247.Death_at_Wentwater_Court',
 'https://www.goodreads.com/book/show/2685034-one-bad-apple',
 'https://www.goodreads.com/book/show/6077869-homicide-in-hardcover',
 'https://www.goodreads.com/book/show/518518.The_Chocolate_Cat_Caper',
 

In [30]:
test_link="https://www.goodreads.com/book/show/16331.Murder_at_the_Vicarage"

In [31]:
r=requests.get(test_link)
soup=BeautifulSoup(r.text,'lxml')

In [32]:
soup.find('h1').text

'Murder at the Vicarage'

In [33]:
Name=soup.find('h1').text
Name

'Murder at the Vicarage'

In [34]:
Contributor_Name= soup.find('span', class_='ContributorLink__name').text
Contributor_Name

'Agatha Christie'

In [35]:
rating_text = soup.find('span', class_='RatingStars RatingStars__medium')['aria-label']
rating = float(rating_text.split()[1])
print(rating)

4.05


In [36]:
rating_count_text = soup.find('span', {'data-testid': 'ratingsCount'}).text
rating_count = int(rating_count_text.replace(',', '').split()[0])
rating_count

187919

In [37]:
reviews_count_text = soup.find('span', class_='u-dot-before').text
reviews_count = int(reviews_count_text.replace(',', '').split()[0])
reviews_count

7364

In [38]:
Format=soup.find('p').text
Format

'288 pages, Hardcover'

In [39]:
product_data = []

for link in product_links:
    try:
        r = requests.get(link)
        soup = BeautifulSoup(r.text, 'lxml')

        try:
            h1_tag = soup.find('h1')
            if h1_tag:
                Name = h1_tag.text.strip()
            else:
                Name = None
        except AttributeError:
            print("Error occurred while scraping 'h1' tag in link:", link)
            Name = None

        try:
            contributor_tag = soup.find('span', class_='ContributorLink__name')
            if contributor_tag:
                Contributor_Name = contributor_tag.text.strip()
            else:
                Contributor_Name = None
        except AttributeError:
            print("Error occurred while scraping 'ContributorLink__name' in link:", link)
            Contributor_Name = None

        rating_text = soup.find('span', class_='RatingStars RatingStars__medium')
        if rating_text:
            try:
                rating = float(rating_text['aria-label'].split()[1])
            except (IndexError, ValueError):
                print("Error occurred while scraping 'rating' in link:", link)
                rating = None
        else:
            print("Error occurred while scraping 'rating' in link:", link)
            rating = None

        try:
            rating_count_text = soup.find('span', {'data-testid': 'ratingsCount'}).text
            rating_count = int(rating_count_text.replace(',', '').split()[0])
        except (AttributeError, ValueError):
            print("Error occurred while scraping 'rating_count' in link:", link)
            rating_count = None

        try:
            reviews_count_text = soup.find('span', class_='u-dot-before').text
            reviews_count = int(reviews_count_text.replace(',', '').split()[0])
        except (AttributeError, ValueError):
            print("Error occurred while scraping 'reviews_count' in link:", link)
            reviews_count = None

        try:
            Format = soup.find('p').text.strip()
        except AttributeError:
            print("Error occurred while scraping 'Format' in link:", link)
            Format = None

        Book = {'Name': Name, 'Author': Contributor_Name, 'rating': rating, 'rating_count': rating_count,
                'reviews_count': reviews_count, 'Format': Format}
        print(Book)
        product_data.append(Book)
    except requests.exceptions.RequestException as e:
        print("Error occurred while making a request to the link:", link)
        print(e)


{'Name': 'The Secret of Chimneys', 'Author': 'Agatha Christie', 'rating': 3.85, 'rating_count': 27387, 'reviews_count': 2152, 'Format': '400 pages, Paperback'}
{'Name': "Mum's the Word", 'Author': 'Kate Collins', 'rating': 3.75, 'rating_count': 5809, 'reviews_count': 453, 'Format': '304 pages, Mass Market Paperback'}
{'Name': 'Body Movers', 'Author': 'Stephanie Bond', 'rating': 3.87, 'rating_count': 14255, 'reviews_count': 728, 'Format': '384 pages, Mass Market Paperback'}
{'Name': 'The Julius House', 'Author': 'Charlaine Harris', 'rating': 3.85, 'rating_count': 19010, 'reviews_count': 823, 'Format': '256 pages, Mass Market Paperback'}
{'Name': 'Tears of the Giraffe', 'Author': 'Alexander McCall Smith', 'rating': 3.98, 'rating_count': 56720, 'reviews_count': 3363, 'Format': '233 pages, Paperback'}
{'Name': 'Still Life', 'Author': 'Louise Penny', 'rating': 3.9, 'rating_count': 247136, 'reviews_count': 19885, 'Format': '293 pages, Mass Market Paperback'}
Error occurred while scraping 'ra

In [40]:
df=pd.DataFrame(product_data)

In [41]:
df

,Name,Author,rating,rating_count,reviews_count,Format
0,The Secret of Chimneys,Agatha Christie,3.85,27387.0,2152.0,"400 pages, Paperback"
1,Mum's the Word,Kate Collins,3.75,5809.0,453.0,"304 pages, Mass Market Paperback"
2,Body Movers,Stephanie Bond,3.87,14255.0,728.0,"384 pages, Mass Market Paperback"
3,The Julius House,Charlaine Harris,3.85,19010.0,823.0,"256 pages, Mass Market Paperback"
4,Tears of the Giraffe,Alexander McCall Smith,3.98,56720.0,3363.0,"233 pages, Paperback"
...,...,...,...,...,...,...
95,Plum Lovin',Janet Evanovich,3.75,56498.0,1845.0,"164 pages, Hardcover"
96,Size 12 Is Not Fat,Meg Cabot,3.76,87846.0,2936.0,"345 pages, Paperback"
97,The Amazing Mrs. Pollifax,Dorothy Gilman,4.18,10933.0,909.0,"176 pages, Mass Market Paperback"
98,The Quiche of Death,M.C. Beaton,3.73,37404.0,4198.0,"246 pages, Mass Market Paperback"


In [42]:
test='https://www.goodreads.com/book/show/512385.The_Trouble_With_Magic?ac=1&from_search=true&qid=4pYf11Lurw&rank=1'
test1="https://www.goodreads.com/book/show/518518.The_Chocolate_Cat_Caper?ac=1&from_search=true&qid=DiZhQizuX4&rank=1"

In [43]:
r1=requests.get(test1)
soup1=BeautifulSoup(r1.text,'html.parser')

In [44]:
r = requests.get(test)
soup = BeautifulSoup(r.text, 'html.parser')

In [45]:
soup.find_all('dd')

[]

In [46]:
pages_format = soup.find('p', {'data-testid': 'pagesFormat'}).text
pages_format

'261 pages, Paperback'

In [47]:
pages_format = soup1.find('p', {'data-testid': 'pagesFormat'}).text
pages_format

'240 pages, Mass Market Paperback'

In [48]:
test3="https://www.goodreads.com/book/show/8173806-this-pen-for-hire"

In [49]:
r=requests.get(test3)
soup4=BeautifulSoup(r.text,'lxml')

In [50]:
soup4.find('div',class_="RatingStatistics__rating")